In [76]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import cv2
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import label_binarize
from itertools import cycle
import splitfolders
import glob
from PIL import Image


if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    for _ in range(2):
        os.chdir(os.path.dirname(os.getcwdb()))


# Limpieza

- Se borran las imágenes vacías (".png") y las eliminadas durante el procesado del "dataframe" de características (en el "notebook" llamado "images_data_processing"), y se actualiza paralelamente ese conjunto de datos quitándole las filas correspondientes a las imágenes "png" mencionadas

- El resultado es que en la carpeta de imágenes quedan tantos archivos ".jpg" como filas hay en el "dataframe" de características

In [77]:
df_images_data = pd.read_csv(r'data\processed\images_data_processed.csv', index_col='Id')
raw_data_path = r'data\raw\images'


In [78]:
len(df_images_data)


5701

In [79]:
# Hay varias imágenes vacías. Todas están en formato "png". Hay que eliminarlas antes que nada
no_delete_extension = '.jpg'
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for file in filenames:
        if no_delete_extension not in file:
            if file != 'info_raw.txt':
                os.remove(f'{dirpath}/{file}')


In [80]:
# Se han eliminado muchas filas del "dataframe", con lo que toca borrar las imágenes correspondientes
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for file in filenames:
        if file[:-4] not in df_images_data.index and file[:-4] != 'info_raw':
            os.remove(f'{dirpath}/{file}')


In [81]:
# Se crea una lista de las imágenes definitivas de la carpeta
filenames_list = []
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    filenames_list += [file[:-4] for file in filenames if file != "info_raw.txt"]

len(filenames_list)


4519

In [82]:
# Asimismo, hay que borrar del "dataframe" los diamantes que se han sacado de la carpeta por no tener imagen
for index in df_images_data.index:
    if index not in filenames_list:
        try:
            df_images_data.drop(index, inplace=True)
        except Exception:
            print(f'El diamante {index} ya se borró durante la limpieza')

len(df_images_data)


4519

In [83]:
# En el "dataframe" de características de las imágenes solo quedan los mismos diamantes que en la carpeta de imágenes "raw".
sorted(df_images_data.index) == sorted(filenames_list)


True

# Colocación de imágenes

In [84]:
image_list = []
for filename in glob.glob(raw_data_path + '\\*.jpg'):
    image_code = filename.split('\\')[3].split('.')[0]
    image = cv2.imread(filename)
    image_list.append((image_code, image))

df_images_data = df_images_data.sort_index()
image_list.sort()

df_images_data['images'] = image_list

df_images_data.head()


,weight (carat),cut quality,color quality,clarity quality,depth (percentage),length (millimeters),width (millimeters),depth (millimeters),images
Id,,,,,,,,,
111000-6305,1.00,4.0,-9.0,1.0,49.350649,8.47,5.39,3.42,"(111000-6305, [[[178 178 178], [178 178 178], ..."
111000-6320,1.01,4.0,5.0,1.0,43.460765,9.39,5.52,3.24,"(111000-6320, [[[177 179 180], [177 179 180], ..."
1490932,0.27,4.0,3.0,5.0,62.857143,3.94,4.46,2.64,"(1490932, [[[180 178 178], [180 178 178], [180..."
1506838,0.70,4.0,1.0,2.0,64.618434,5.28,4.81,3.26,"(1506838, [[[180 175 176], [180 175 176], [180..."
1513169,0.29,2.0,4.0,2.0,61.899179,4.23,4.30,2.64,"(1513169, [[[182 178 177], [182 178 177], [183..."
